In [10]:
import os, sys
import numpy as np
import scipy as sp
from scipy.interpolate import interp1d
import scipy.integrate as integ
import scipy.optimize as opt
from scipy.optimize import curve_fit
import pandas as pd
import astropy
from astropy.cosmology import FlatLambdaCDM
from lmfit import minimize, Minimizer, Parameters, Parameter, report_fit
import astropy.units as u
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt



In [11]:
cosmo = FlatLambdaCDM(H0=67.7, Om0=0.307)
R_range=np.arange(11.0,18.0,0.1)
T_range=np.arange(3000,50000,100)
CGS_H=6.6260755e-27
CGS_C=2.99792458e10
CGS_K=1.380658e-16
#redshift of your target
redshift=0.016995

In [15]:
def piv_llambda(FILTER):
    Wave2 = integ.trapz(FILTER[1]*FILTER[0],x=FILTER[0])/integ.trapz(FILTER[1],x=FILTER[0])
    return Wave2
loaddict={
    "Bessel_B_0": np.loadtxt('./filters/Bessel_B_0.txt',unpack=True),
    "Bessel_V_0": np.loadtxt('./filters/Bessel_V_0.txt',unpack=True),
    "Bessel_R_0": np.loadtxt('./filters/Bessel_R_0.txt',unpack=True)
}

fdict={"B": "Bessel_B_0",
       "V": "Bessel_V_0",
       "R": "Bessel_R_0",
      }

def BB_fit(T, R, z,wav):
    lumDis = cosmo.luminosity_distance(z).value * 3.08567758e24
    sed = np.zeros(wav.size)
    sed = (2.0 * CGS_H * np.pi * CGS_C**2.0) / (wav * 1e-8)**5
    sed /= np.exp(CGS_H * CGS_C / (wav * 1e-8 * CGS_K * T)) - 1.0
    sed *= 1e-8
    sed *= R**2
    sed /= (lumDis**2)
    sed /= (1+z)
    return sed

def chisq(p, z, bands, photom, photomerr):
    T=p['T']
    R=p['R']
    guess=[]
    filt=[loaddict[i] for i in bands]
    for j in filt:
        BB=BB_fit(T,R,z,j[0]/(1+z))[1]*j[1]
        point=np.trapz(BB,x=j[0])/np.trapz(j[1],j[0])
        guess.append(point)
    chi2=(((guess-photom)/photomerr))
    return chi2
        

OSError: ./filters/Bessel_B_0.txt not found.